# Homework Assignment

Upgrade Day 1 summary project to summarize webpage using ollama.

## Recap on installation of Ollama

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  



## Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

## Constants

In [2]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

## Initial Setup

In [3]:
# Create a message list using same format as OpenAI

messages = [
    {
        "role": "user",
        "content": "Describe some of the business applications of Generative AI"
    }
]

In [4]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

In [5]:
# Let's just be sure model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# Now to try

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as blog posts, social media posts, product descriptions, and even entire articles. This can save time and resources for content creation teams.
2. **Product Design and Development**: Generative AI can aid in designing new products, such as clothing, furniture, or electronics, by generating 3D models and prototypes. It can also help with product development, testing, and optimization.
3. **Marketing Automation**: Generative AI can be used to personalize marketing messages and offers for individual customers based on their behavior, preferences, and demographics. This can lead to increased conversion rates and improved customer engagement.
4. **Chatbots and Virtual Assistants**: Generative AI can power chatbots and virtual assistants that can understand natural language queries, respond with relevan

## The Ollama Package

We'll do same thing but using ollama python package instead of direct HTTP call

In [7]:
import ollama

In [8]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as articles, social media posts, and product descriptions. This can help businesses reduce the time and cost associated with content creation.
2. **Product Design and Development**: Generative AI can be used to design new products, such as furniture, electronics, or fashion items. It can also be used to generate 3D models, prototypes, and even entire product lines.
3. **Image and Video Generation**: Generative AI can be used to create realistic images and videos that can be used for advertising, marketing, and other purposes. This can help businesses create engaging visual content without the need for expensive photography or video production.
4. **Music and Audio Generation**: Generative AI can be used to create original music, sound effects, and audio tracks. This can be useful for businesses th

## Now for the Webpage Summarizer

### Create Webpage Scraper Class

In [9]:
# Class to represent webpage

# Some websites need you to use proper headers when fetching them
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """Create this Website object from given url using bs4"""
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

### System Prompt

In [10]:
# Define system prompt

system_prompt = "You are an assistant that analyzes the contents of a website " \
"and provides a short summary, ignoring text that might be navigation-related. " \
"Respond in markdown."

### User Prompt Function

In [11]:
# A function that writes a User Prompt that asks for summaries of websites

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website are as follows: "
    "please provide a short summary of this website in markdown."
    "If it includes news or announcements, summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

### Messages Function

In [12]:
# Function to create and format messages param

def messages_for(website):
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt_for(website)
        }
    ]

### Summarize Function

In [13]:
# Now call ollama api

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=messages_for(website)
    )
    return response['message']['content']

### Display Format Function

In [14]:
# Display format function

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

## Testing Functions

Wanted all functions for setup all together with no interruption

In [15]:
# Test summarize

summarize("https://edwarddonner.com")

'**Summary**\nThis website belongs to Edward Donner, the co-founder and CTO of Nebula.io. The site appears to be a personal blog or portfolio, showcasing his work in AI and talent management. It includes sections on "About", "Posts" (which seem to be resources for learning about AI and LLMs), and a newsletter sign-up.\n\n**Key Points**\n\n* Edward Donner is the co-founder and CTO of Nebula.io, which applies AI to help people discover their potential.\n* He has experience as the founder and CEO of another AI startup, untapt, which was acquired in 2021.\n* The website includes resources for learning about AI and LLMs, such as workshops and tutorials.'

In [16]:
# Test display function
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

## Overview

This website appears to be the personal blog or portfolio of Edward Donner, a technologist and entrepreneur with experience in AI development and deployment. The site showcases his work on various projects and initiatives related to artificial intelligence.

## Key Takeaways

* Edward Donner is the co-founder and CTO of Nebula.io, an AI company working to help people discover their potential.
* He has previously founded and led AI startups, including untapt, which was acquired in 2021.
* The website features articles about LLMs (Large Language Models), AI engineering, and related topics.
* There are resources available for developers interested in mastering AI and LLMs.

## Notable Links

* **Connect Four**: A game that pits LLMs against each other in a battle of diplomacy and deviousness
* **About**: Edward Donner's introduction, including his interests and experience
* **Newsletter**: Subscription link to receive updates from the website author

In [17]:
display_summary("https://cnn.com")

There are several interesting articles and topics on CNN's website today, but I'll highlight a few:

1. **Deadly earthquake strikes Myanmar, Thailand, and China**: A powerful earthquake has struck the border region of Myanmar, Thailand, and China, causing widespread destruction and loss of life.
2. **Val Kilmer recently gave us ‘Batman’ one last time**: Val Kilmer, who played Batman in the 1995 film "Batman Forever," recently reprised his role in a new documentary about the character's history.
3. **Lawyer for Trump co-defendant and alleged US Capitol rioters picked for major role at DOJ**: A lawyer who represented several individuals involved in the January 6th US Capitol riot has been appointed to a key position at the Department of Justice.
4. **A lead crisis in schools led a city to reach out to CDC for help**: A city is seeking assistance from the Centers for Disease Control and Prevention (CDC) due to a lead contamination crisis in its schools, which may have affected hundreds of children.

These are just a few examples of the many interesting articles and topics on CNN's website today.